In [1]:
from tensorflow.keras.utils import plot_model
import tensorflow.keras as keras 


### ANN model for sine fitting

In [2]:
class SineModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.hidden1 = keras.layers.Dense(40, input_shape=(1,))
        self.hidden2 = keras.layers.Dense(40)
        self.out = keras.layers.Dense(1) 
    
    def forward(self, x):
        x = keras.activations.relu(self.hidden1(x))
        x = keras.activations.relu(self.hidden2(x))
        x = self.out(x) 
        return x 

### Sinusoid Generator

In [3]:
import numpy as np 
import matplotlib.pyplot as plt 

class SinusoidGenerator():
    ''' 
    Sinusoid Generator.

    p(T) is continuous, where the amplitude varies within [0.1, 5.0]
    and the phase varies within [0, pi]
    '''

    def __init__(self, K=10, amplitude=None, phase=None):
        ''' 
        Args:
        K: batch size. Number of values sampled at every batch
        amplitude: Sine wave Amplitude. If None, uniformly sampled from [0.1, 5.0]
        phase: Sine wave phase. If None, uniformly sample from [0, pi] interval
        '''
        self.K = K 
        self.amplitude = amplitude if amplitude else np.random.uniform(0.1, 5.0)
        self.phase = phase if amplitude else np.random.uniform(0, np.pi)
        self.sampled_points = None 
        self.x = self._sample_x() 
    
    def _sample_x(self):
        return np.random.uniform(-5, 5, self.K) 

    def f(self, x):
        '''Sine wave function'''
        return self.amplitude * np.sin(x - self.phase) 

    def batch(self, x = None, force_new=False):
        print(x)
        if x is None:
            if force_new:
                x = self._sample_x() 
            else: 
                x = self.x 
        y = self.f(x) 
        return x[:, None], y[:, None] 

    def equally_spaced_samples(self, K=None):
        """Returns `K` equally spaced samples"""
        if K is None:
            K = self.K 
        return self.batch(x=np.linspace(-5, 5, K))

### Util Functions

In [4]:
def plot(data, *args, **kwargs):
    x, y = data 
    return plt.plot(x, y, *args, **kwargs) 

def generate_dataset(K, train_size=20000, test_size=10):
    def _generate_dataset(size):
        return [SinusoidGenerator(K=K) for _ in range(size)]
    return _generate_dataset(train_size), _generate_dataset(test_size)

### Train Sine Model 

In [6]:
import os 
import sys 
import random 
import numpy as np 
import tensorflow as tf 
import time 
import matplotlib.pyplot as plt 

tf.keras.backend.set_floatx('float64')

def np_to_tensor(list_of_numpy_objs):
    return (tf.convert_to_tensor(obj) for obj in list_of_numpy_objs)

def copy_model(mode, x):
    copied_model = SineModel() 
    copied_model.forward(x) 
    copied_model.set_weights(mode.get_weights())
    return copied_model 

def loss_fn(y, pred_y):
    return tf.reduce_mean(tf.keras.metrics.mean_squared_error(y, pred_y))

def compute_loss(model, x, y, loss_fn=loss_fn):
    logits = model.forward(x) 
    mse = loss_fn(logits, y) 
    return mse, logits 

def compute_gradients(model, x, y, loss_fn=loss_fn):
    with tf.GradientTape() as tape:
        loss, logits = compute_loss(model, x, y, loss_fn) 
    return tape.gradient(loss, model.trainable_variables), loss 

def apply_gradients(optimizer, gradients, variables):
    optimizer.apply_gradients(zip(gradients, variables))

def train_step(x, y, model, optimizer):
    tensor_x, tensor_y = np_to_tensor((x, y))
    gradients, loss = compute_gradients(model, tensor_x, tensor_y)
    apply_gradients(optimizer, gradients, model.trainable_variables)
    return loss 
